# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117844e+02     1.600886e+00
 * time: 0.09354519844055176
     1     1.100442e+01     9.071146e-01
 * time: 0.9407131671905518
     2    -1.162130e+01     1.005346e+00
 * time: 1.0902221202850342
     3    -3.433736e+01     7.201658e-01
 * time: 1.2647461891174316
     4    -4.784919e+01     4.956766e-01
 * time: 1.4197101593017578
     5    -5.714380e+01     2.042683e-01
 * time: 1.5699481964111328
     6    -5.997586e+01     8.359153e-02
 * time: 1.6802730560302734
     7    -6.091982e+01     5.674456e-02
 * time: 1.799307107925415
     8    -6.130246e+01     6.795625e-02
 * time: 1.9168310165405273
     9    -6.162575e+01     3.921131e-02
 * time: 2.0280210971832275
    10    -6.184790e+01     2.703787e-02
 * time: 2.146329164505005
    11    -6.200163e+01     2.271938e-02
 * time: 2.2597811222076416
    12    -6.207788e+01     1.874001e-02
 * time: 2.3836450576782227
    13    -6.213835e+01     1.493370e-02
 * time: 2.50178003311

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557645
    AtomicNonlocal      14.8522614
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485364 
    Xc                  -19.3336817

    total               -62.261666459984
